In [2]:
# this shall train neural network to recognize category / words ( total 30 ) from 1 second audio
# input audio format : format 16-bit little-endian PCM-encoded WAVE file at a 16000 sample rate

import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import scipy.io.wavfile as wav
from python_speech_features import mfcc


In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
    #d_mfcc_feat = delta(mfcc_feat, 2)
    #fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
# test above function
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [6]:
# Training Parameters
learning_rate = 0.001
training_steps = 20
batch_size = 2
display_step = 200
num_features = 13

# Network Parameters
num_input = 28 #  data input (img shape: 28*28) # need to investigate this , shouldn't it be = num_features = 13 ?
timesteps = 300 # maximum timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 30   # since there are 30 categories 

In [7]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [13]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [10]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    # get ony the output of last rnn timestep
    outputs=outputs[-1]
    
    
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    return res

In [11]:
inputs = tf.placeholder(tf.float32, [None, None, num_features])

In [14]:
logits = RNN(inputs, weights, biases)

ValueError: Shape must be rank 2 but is rank 3 for 'MatMul' (op: 'MatMul') with input shapes: [?,256], [2,256,30].